In [ ]:
"""Sort portfolios into quintiles by several (high-frequency and low-frequency)
liquidity measures to examine illiquidity premium.
"""
import glob
import os

import pandas as pd
import numpy as np
import jdatetime

pd.options.mode.chained_assignment = None

In [ ]:
# Read "Bourseview" data for market cap
# Concat all 75 tickers' data
bv_list = []
for file_number in range(1, 76):
    bv_path = f'E:/Thesis/New Sampling/Daily Data - Bourseview/'\
        f'{file_number}.xlsx'
    bv_df = pd.read_excel(
        bv_path,
        skiprows=7,
        usecols=[2, 3, 11],
        names=['date', 'open', 'market_cap'],
        na_values='-'
    )
    # Change order from old to new dates
    bv_df = bv_df[::-1].reset_index(drop=True)
    bv_df['date'] = bv_df['date'].str.replace('-', '')
    # Delete non-traded days
    bv_df.dropna(subset=['open'], inplace=True)
    bv_df.drop(columns='open', inplace=True)
    # Create monthly dataframe for each ticker
    bv_df = bv_df.groupby(bv_df['date'].str[:6]).last()
    bv_df = bv_df.drop(columns=['date']).reset_index()
    bv_df.insert(1, 'ticker_num', file_number)
    bv_list.append(bv_df)
# Create monthly dataframe of market cap that contains all tickers
bv_df = pd.concat(bv_list, ignore_index=True)
bv_df = bv_df.loc[(bv_df['date'] >= '139300') & (bv_df['date'] <= '139900')]
bv_df.reset_index(drop=True, inplace=True)

In [ ]:
# Read "rahavard 365" data for calculating returns
# Concat all 75 tickers' data
daily_list = []
monthly_list = []
for file_number in range(1, 76):
    rahavard_path = f'E:/Thesis/New Sampling/Daily Data - Rahavard 365/'\
        f'{file_number}.txt'
    df = pd.read_csv(
        rahavard_path,
        usecols=[2, 7, 8],
        names=['date', 'adj_close', 'd_vol'],
        header=0,
        dtype={'date': str},
        parse_dates=[0]
    )
    # Solve index reading problem, pandas add 2 index to the df
    df.reset_index(drop=True, inplace=True)
    # Convert to persian dates
    df['date'] = df['date'].apply(
        lambda x: jdatetime.date.fromgregorian(date=x).strftime('%Y%m%d')
    )
    df.insert(1, 'ticker_num', file_number)
    df['i_return'] = df['adj_close'].pct_change()
    daily_list.append(df)
    # Create monthly return dataframe for each ticker
    monthly_df = df.groupby(df['date'].str[:6]).last()
    monthly_df = monthly_df.drop(columns=['date']).reset_index()
    monthly_df['monthly_return'] = monthly_df['adj_close'].pct_change()
    monthly_df.drop(columns=['d_vol', 'i_return'], inplace=True)
    monthly_list.append(monthly_df)
# Create monthly return dataframe that contains all tickers
monthly_df = pd.concat(monthly_list, ignore_index=True)
monthly_df = monthly_df.loc[(
    monthly_df['date'] >= '139300') & (monthly_df['date'] <= '139900')
]
monthly_df.dropna(inplace=True)
monthly_df.reset_index(drop=True, inplace=True)

In [ ]:
# Extract all months from index df
index_path = r'E:\Thesis\New Sampling\TEDPIX\شاخص كل6.xls'
index_df = pd.read_excel(
    index_path,
    usecols=[1, 3],
    names=['date', 'close'],
    dtype={'date': str}
)
index_df.dropna(inplace=True)
index_df['m_return'] = index_df['close'].pct_change()
all_months = pd.Series(index_df['date'].str[:6].unique().tolist())
all_months.name = 'date'

In [ ]:
# Create data frame that contains all tickers' max and mean spread
max_path = r'E:\Thesis\New Sampling\Max Spreads'
all_files = glob.glob(os.path.join(max_path, '*.csv'))
factor_dfs = (pd.read_csv(f) for f in all_files)
factor_df = pd.concat(factor_dfs, ignore_index=True)
factor_df.rename(
    columns={'tricker_num': 'ticker_num', 'j_date': 'date'},
    inplace=True
)
factor_df['date'] = factor_df['date'].astype(str)

In [ ]:
# ّّFirst case: calculate equal-weighted portfolio returns
# return_df = monthly_df

# ّّSecond case: calculate value-weighted portfolio returns
# Merge market cap and price dfs
return_df = pd.merge(monthly_df, bv_df, on=['ticker_num', 'date'])

In [ ]:
# Sort data frames by ticker number and date
factor_df = factor_df.sort_values(
    by=['ticker_num', 'date'],
    ascending=[True, True],
    ignore_index=True
)
return_df = return_df.sort_values(
    by=['ticker_num', 'date'],
    ascending=[True, True],
    ignore_index=True
)

In [ ]:
# Create a list of factors
factors = factor_df.columns[2:]
factors

In [ ]:
# Create a list of all months
months = all_months.loc[
    (all_months >= '139400') & (all_months <= '139900')
].tolist()

In [ ]:
# ّّFirst case: calculate equal-weighted portfolio returns
# Make equal-weighted portfolios sorted into quintiles
# by each factor in month t-1
max_mean_list = []
for factor in factors:
    for month in months:
        all_months_list = all_months.tolist()
        # Create dataframe that includes factor in the previous month
        previous_month = all_months_list[all_months_list.index(month) - 1]
        prev_month_condition = (factor_df['date'] == previous_month)
        sorting_df = factor_df.loc[
            prev_month_condition
        ][['date', 'ticker_num', factor]]
        # sort into quintiles
        quantiles = pd.qcut(
            sorting_df[factor],
            q=5,
            labels=[1, 2, 3, 4, 5]
        ).to_dict()
        sorting_df['quantile'] = pd.Series(quantiles)
        q1 = sorting_df.loc[
        sorting_df['quantile'] == 1
        ]['ticker_num'].tolist()
        q2 = sorting_df.loc[
        sorting_df['quantile'] == 2
        ]['ticker_num'].tolist()
        q3 = sorting_df.loc[
        sorting_df['quantile'] == 3
        ]['ticker_num'].tolist()
        q4 = sorting_df.loc[
        sorting_df['quantile'] == 4
        ]['ticker_num'].tolist()
        q5 = sorting_df.loc[
            sorting_df['quantile'] == 5
        ]['ticker_num'].tolist()
        # Create portfolio of stocks in month t
        # based on above-mentioned conditions
        q1_condition = return_df['ticker_num'].isin(q1)
        q2_condition = return_df['ticker_num'].isin(q2)
        q3_condition = return_df['ticker_num'].isin(q3)
        q4_condition = return_df['ticker_num'].isin(q4)
        q5_condition = return_df['ticker_num'].isin(q5)
        month_condition = (return_df['date'] == month)
        q1_portfolio = return_df.loc[q1_condition & month_condition]
        q2_portfolio = return_df.loc[q2_condition & month_condition]
        q3_portfolio = return_df.loc[q3_condition & month_condition]
        q4_portfolio = return_df.loc[q4_condition & month_condition]
        q5_portfolio = return_df.loc[q5_condition & month_condition]
        # Calculate equal-weighted portfolio returns
        q1_return = np.average(q1_portfolio.monthly_return)
        q2_return = np.average(q2_portfolio.monthly_return)
        q3_return = np.average(q3_portfolio.monthly_return)
        q4_return = np.average(q4_portfolio.monthly_return)
        q5_return = np.average(q5_portfolio.monthly_return)
        max_mean_list.append([
            factor, month,
            q1_return, q2_return, q3_return, q4_return, q5_return
        ])

In [ ]:
# ّّSecond case: calculate value-weighted portfolio returns
# Make value-weighted portfolios sorted into quintiles
# by each factor in month t-1
max_mean_list = []
for factor in factors:
    for month in months:
        all_months_list = all_months.tolist()
        # Create dataframe that includes factor in the previous month
        previous_month = all_months_list[all_months_list.index(month) - 1]
        prev_month_condition = (factor_df['date'] == previous_month)
        sorting_df = factor_df.loc[
            prev_month_condition
        ][['date', 'ticker_num', factor]]
        # sort into quintiles
        quantiles = pd.qcut(
            sorting_df[factor],
            q=5,
            labels=[1, 2, 3, 4, 5]
        ).to_dict()
        sorting_df['quantile'] = pd.Series(quantiles)
        q1 = sorting_df.loc[
        sorting_df['quantile'] == 1
        ]['ticker_num'].tolist()
        q2 = sorting_df.loc[
        sorting_df['quantile'] == 2
        ]['ticker_num'].tolist()
        q3 = sorting_df.loc[
        sorting_df['quantile'] == 3
        ]['ticker_num'].tolist()
        q4 = sorting_df.loc[
        sorting_df['quantile'] == 4
        ]['ticker_num'].tolist()
        q5 = sorting_df.loc[
            sorting_df['quantile'] == 5
        ]['ticker_num'].tolist()
        # Create portfolio of stocks in month t
        # based on above-mentioned conditions
        q1_condition = return_df['ticker_num'].isin(q1)
        q2_condition = return_df['ticker_num'].isin(q2)
        q3_condition = return_df['ticker_num'].isin(q3)
        q4_condition = return_df['ticker_num'].isin(q4)
        q5_condition = return_df['ticker_num'].isin(q5)
        month_condition = (return_df['date'] == month)
        q1_portfolio = return_df.loc[q1_condition & month_condition]
        q2_portfolio = return_df.loc[q2_condition & month_condition]
        q3_portfolio = return_df.loc[q3_condition & month_condition]
        q4_portfolio = return_df.loc[q4_condition & month_condition]
        q5_portfolio = return_df.loc[q5_condition & month_condition]
        # Calculate value-weighted portfolio returns
        q1_return = np.average(
            q1_portfolio.monthly_return,
            weights=q1_portfolio.market_cap
        )
        q2_return = np.average(
            q2_portfolio.monthly_return,
            weights=q2_portfolio.market_cap
        )
        q3_return = np.average(
            q3_portfolio.monthly_return,
            weights=q3_portfolio.market_cap
        )
        q4_return = np.average(
            q4_portfolio.monthly_return,
            weights=q4_portfolio.market_cap
        )
        q5_return = np.average(
            q5_portfolio.monthly_return,
            weights=q5_portfolio.market_cap
        )
        max_mean_list.append([
            factor, month,
            q1_return, q2_return, q2_return, q4_return, q5_return
        ])

In [ ]:
# Create data frame of quintiles for each month, and write it to excel
res_df = pd.DataFrame(
    max_mean_list,
    columns=['factor', 'month', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5']
)
res_df.to_excel('spread_quintiles.xlsx', index=False)

In [ ]:
# Calculate the difference in returns between extreme portfolios
# and corresponding t-statistic
diff_list = []
for factor in factors:
    factor_df = res_df[res_df['factor'] == factor]
    factor_df['diff'] = factor_df['Q5'] - factor_df['Q1']
    diff_mean = factor_df['diff'].mean()
    diff_std = factor_df['diff'].std()
    diff_sem = factor_df['diff'].sem()
    t_statistic = diff_mean/diff_sem
    diff_list.append([
        factor,
        factor_df['Q1'].mean(),
        factor_df['Q2'].mean(),
        factor_df['Q3'].mean(),
        factor_df['Q4'].mean(),
        factor_df['Q5'].mean(),
        diff_mean,
        t_statistic
    ])
    print(factor, diff_mean, t_statistic)

In [ ]:
# Write calculated means and t-statistic to excel
diff_df = pd.DataFrame(
    diff_list,
    columns=['factor', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q5-Q1', 't-statistic']
)
diff_df.to_excel('spread_portfolio_analysis.xlsx', index=False)